# Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn import model_selection

In [2]:
# получение датасета
df = pd.read_csv("../data/weather_preprocessed.csv")

In [4]:
# первые 5 записей из датасета
df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,WindDir3pm_without_value
0,13.4,22.9,0.6,5.468232,7.611178,44.0,20.0,24.0,71.0,22.0,...,0,0,0,0,0,0,0,1,0,0
1,7.4,25.1,0.0,5.468232,7.611178,44.0,4.0,22.0,44.0,25.0,...,0,0,0,0,0,0,0,0,1,0
2,12.9,25.7,0.0,5.468232,7.611178,46.0,19.0,26.0,38.0,30.0,...,0,0,0,0,0,0,0,0,1,0
3,9.2,28.0,0.0,5.468232,7.611178,24.0,11.0,9.0,45.0,16.0,...,0,0,0,0,0,0,0,0,0,0
4,17.5,32.3,1.0,5.468232,7.611178,41.0,7.0,20.0,82.0,33.0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
# информация о датасете 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140787 entries, 0 to 140786
Columns: 121 entries, MinTemp to WindDir3pm_without_value
dtypes: float64(16), int64(105)
memory usage: 130.0 MB


In [5]:
# колиество нулевых элементов
df.isnull().sum()

MinTemp                     0
MaxTemp                     0
Rainfall                    0
Evaporation                 0
Sunshine                    0
                           ..
WindDir3pm_SW               0
WindDir3pm_W                0
WindDir3pm_WNW              0
WindDir3pm_WSW              0
WindDir3pm_without_value    0
Length: 121, dtype: int64

In [6]:
# разделение признаков
y = df['RainTomorrow'].values
df.pop('RainTomorrow')
X = df.values

In [7]:
# разделение датасета на обучаемую и тестовую выборки
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

In [8]:
# Масштабирование признаков
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)